In [38]:
import pandas as pd

filepath = r"C:\Users\usuario\Documents\ALBERTO\IPS\ips_dashboard\data\output\SC_BLE_Fingerprinting\Calibration_21112023\Calibration_21112023.csv"

df = pd.read_csv(filepath)

df.describe()

,timestamp,x,y,channel,rssi
count,3.118180e+05,311818.000000,311818.000000,311818.000000,311818.000000
mean,1.700580e+12,0.000898,0.000558,37.999416,-42.830610
std,8.368222e+06,3.316312,3.741436,0.816379,7.076597
min,1.700565e+12,-5.000000,-6.000000,37.000000,-81.000000
25%,1.700573e+12,-3.000000,-3.000000,37.000000,-47.000000
50%,1.700580e+12,1.000000,0.000000,38.000000,-42.000000
75%,1.700588e+12,3.000000,3.000000,39.000000,-38.000000
max,1.700595e+12,5.000000,6.000000,39.000000,-22.000000


In [39]:
df['beacon']

0         P6
1         P5
2         P7
3         P4
4         P2
          ..
311813    P8
311814    P3
311815    P7
311816    P3
311817    P7
Name: beacon, Length: 311818, dtype: object

In [40]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
from scipy.interpolate import griddata
from PIL import Image
import base64
from io import BytesIO

In [41]:
# # demo data
# puntos = np.array([
#     [-1, -1, -10],
#     [-1, +0, -20],
#     [-1, +1, -35],
#     [+0, -1, -45],
#     [+0, +0, -25],
#     [+0, +1, -30],
#     [+1, -1, -55],
#     [+1, +0, -40],
#     [+1, +1, -50]
# ])

# select data
df_filtered = df[(df["channel"] == 37) & (df["beacon"] == "P2")]

# tomar la primera aparicion por coordenada (x, y)
df_filtered = df_filtered.sort_index().drop_duplicates(subset=["x", "y"], keep="first")
puntos = df_filtered[["x", "y", "rssi"]].values
puntos

array([[ -1,  -1, -38],
       [ -1,  -2, -34],
       [ -1,  -3, -35],
       [ -1,  -4, -39],
       [ -1,  -5, -45],
       [ -1,  -6, -45],
       [ -1,   0, -40],
       [ -1,   1, -43],
       [ -1,   2, -47],
       [ -1,   3, -43],
       [ -1,   4, -51],
       [ -1,   5, -52],
       [ -1,   6, -52],
       [ -2,  -1, -39],
       [ -2,  -2, -35],
       [ -2,  -3, -34],
       [ -2,  -4, -40],
       [ -2,  -5, -40],
       [ -2,  -6, -58],
       [ -2,   0, -40],
       [ -2,   1, -45],
       [ -2,   2, -58],
       [ -2,   3, -50],
       [ -2,   4, -48],
       [ -2,   5, -50],
       [ -2,   6, -48],
       [ -3,  -1, -39],
       [ -3,  -2, -37],
       [ -3,  -3, -36],
       [ -3,  -4, -35],
       [ -3,  -5, -44],
       [ -3,  -6, -41],
       [ -3,   0, -49],
       [ -3,   1, -42],
       [ -3,   2, -48],
       [ -3,   3, -48],
       [ -3,   4, -60],
       [ -3,   5, -48],
       [ -3,   6, -51],
       [ -4,  -1, -35],
       [ -4,  -2, -36],
       [ -4,  -3

In [ ]:
def toImgCoord(x, y, width=100, height=100, margin_x=0, margin_y=0):
    x = np.asarray(x)
    y = np.asarray(y)
    
    xrange = (x.min(), x.max())
    yrange = (y.min(), y.max())

    x_den = xrange[1] - xrange[0]
    y_den = yrange[1] - yrange[0]

    x_norm = np.zeros_like(x, dtype=float)
    y_norm = np.zeros_like(y, dtype=float)

    if x_den != 0:
        x_norm = (x - xrange[0]) / x_den
    if y_den != 0:
        y_norm = (y - yrange[0]) / y_den

    draw_width = max(width - 2 * margin_x, 1)
    draw_height = max(height - 2 * margin_y, 1)

    x_px = (margin_x + x_norm * draw_width).astype(int)
    y_px = (margin_y + y_norm * draw_height).astype(int)

    return x_px, y_px

def create_heatmap(points):
    # ========================================
    # Convertir imagen/mapa a base64
    # ========================================
    img_path = "room.png"
    img = Image.open(img_path)
    width, height = img.size
    
    # Convertir imagen a base64 para Plotly
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    img_base64 = "data:image/png;base64," + base64.b64encode(buffer.getvalue()).decode()
    
    # ========================================
    # Convertir coordenadas a px imagen
    # ========================================
    x = points[:, 0]
    y = points[:, 1]
    rssi = points[:, 2]

    margin_x = 20
    margin_y = 20
    x_px, y_px = toImgCoord(x, y, height=height, width=width, margin_x=margin_x, margin_y=margin_y)

    # ========================================
    # Interpolacion + suavizado gaussiano
    # ========================================
    resolution = 300  # resolucion del grid

    xi = np.linspace(0, width, resolution)
    yi = np.linspace(0, height, resolution)
    xi_grid, yi_grid = np.meshgrid(xi, yi)

    # Rango RSSI fijo para colores y normalizacion
    zmin, zmax = -80.0, 0.0

    # Interpolar valores en todo el grid
    zi = griddata((x_px, y_px), rssi, (xi_grid, yi_grid), method="cubic", fill_value=zmin)

    # Aplicar suavizado gaussiano (sigma alto = mas difuminado, estilo eye-tracking)
    sigma = 10  # Ajusta: mas alto -> mas suave y difuminado
    zi_smooth = gaussian_filter(zi, sigma=sigma)

    # Normalizar para mascara de transparencia
    zi_norm = (zi_smooth - zmin) / (zmax - zmin)
    zi_norm = np.clip(zi_norm, 0, 1)

    # ========================================
    # Crear mascara de transparencia (ocultar zonas sin datos)
    # ========================================
    # Las zonas con valor muy bajo se hacen transparentes
    threshold_norm = 0.05
    zi_display = np.where(zi_norm > threshold_norm, zi_smooth, np.nan)

    # Texto de hover con control de NaN
    hover_text = np.empty_like(zi_display, dtype=object)
    mask = np.isnan(zi_display)
    hover_text[mask] = "RSSI: sin datos"
    hover_text[~mask] = np.array([f"RSSI: {v:.1f} dBm" for v in zi_display[~mask]])

    # ========================================
    # Colorscale personalizada estilo eye-tracking
    # ========================================
    colorscale_eyetracking = [
        [0.0, "rgba(0, 128, 0, 0.0)"],      # Transparente
        [0.15, "rgba(0, 128, 0, 0.3)"],      # Verde suave
        [0.3, "rgba(0, 200, 0, 0.5)"],       # Verde
        [0.45, "rgba(144, 238, 0, 0.6)"],    # Verde-amarillo
        [0.6, "rgba(255, 255, 0, 0.7)"],     # Amarillo
        [0.75, "rgba(255, 165, 0, 0.8)"],    # Naranja
        [0.9, "rgba(255, 69, 0, 0.85)"],     # Rojo-naranja
        [1.0, "rgba(255, 0, 0, 0.9)"],       # Rojo intenso
    ]
    
    # ========================================
    # Crear figura
    # ========================================

    fig = go.Figure()

    # Imagen de fondo (plano)
    fig.add_layout_image(
        dict(
            source=img_base64,
            x=0, y=0,
            sizex=width, sizey=height,
            xref="x", yref="y",
            sizing="stretch",
            layer="below",
            opacity=1.0,
        )
    )

    # Heatmap suavizado estilo eye-tracking
    fig.add_trace(go.Heatmap(
        x=xi,
        y=yi,
        z=zi_display,
        text=hover_text,
        colorscale=colorscale_eyetracking,
        zmin=zmin, zmax=zmax,
        opacity=0.7,
        showscale=True,
        colorbar=dict(
            title=dict(text="RSSI (dBm)", side="right"),
            thickness=15,
            len=0.6,
            tickvals=[-80, -60, -40, -20, 0],
            ticktext=["-80", "-60", "-40", "-20", "0"],
            x=1.02,
            xanchor="left",
            y=0.5,
        ),
        hovertemplate=(
            "X: %{x:.0f} px<br>"
            "Y: %{y:.0f} px<br>"
            "%{text}<extra></extra>"
        ),
        connectgaps=False,
    ))

    # Puntos de medicion (pintados en px, mostrando coordenada real)
    customdata = np.column_stack([x, y, rssi])
    fig.add_trace(go.Scatter(
        x=x_px, y=y_px,
        mode="markers+text",
        marker=dict(
            size=12,
            color="white",
            symbol="circle",
            line=dict(width=2, color="black"),
        ),
        text=[f"{v:.2f} dBm" for v in rssi],
        textposition="top center",
        textfont=dict(size=10, color="black"),
        name="Puntos de medicion",
        customdata=customdata,
        hovertemplate=(
            "Punto de medicion<br>"
            "X: %{customdata[0]:.2f}<br>"
            "Y: %{customdata[1]:.2f}<br>"
            "RSSI: %{customdata[2]:.1f} dBm<extra></extra>"
        ),
    ))

    fig.update_layout(
        title=dict(text=""),
        xaxis=dict(
            range=[0, width],
            visible=False,
            scaleanchor="y",
            constrain="domain",
            domain=[0, 1],
        ),
        yaxis=dict(
            range=[height, 0],  # Invertir Y para que coincida con imagen
            visible=False,
            constrain="domain",
            domain=[0, 1],
        ),
        width=900,
        height=int(900 * height / width),
        margin=dict(l=0, r=0, t=10, b=0),
        plot_bgcolor="white",
        # Botones interactivos
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                x=0.7, y=1.1,
                pad=dict(t=10),
                buttons=[
                    dict(
                        label="Mostrar puntos",
                        method="update",
                        args=[{"visible": [True, True]}],
                    ),
                    dict(
                        label="Solo heatmap",
                        method="update",
                        args=[{"visible": [True, False]}],
                    ),
                ],
            )
        ],
        # Slider para opacidad
        sliders=[
            dict(
                active=7,
                currentvalue=dict(prefix="Opacidad: ", suffix=""),
                pad=dict(t=0, b=15),
                x=0.05, y=0.0,
                steps=[
                    dict(
                        method="restyle",
                        args=["opacity", [o / 10]],
                        label=f"{o / 10:.1f}",
                    )
                    for o in range(0, 11)
                ],
            )
        ],
    )

    return fig


In [43]:
fig = create_heatmap(puntos)
fig.show()